In [11]:
# ==============================================================================
#  PYTHON-ONLY MOBILITY DATA GENERATOR (NO SUMO REQUIRED)
# ==============================================================================
#
#  This script simulates vehicle movement on a virtual grid to generate the
#  'user_mobility.csv' file without any external dependencies.
#
#  How it works:
#  1. A Vehicle class manages the state of each user (position, speed, direction).
#  2. Vehicles are placed on a virtual grid (e.g., roads every 1000m).
#  3. At each timestep, vehicles move forward.
#  4. When a vehicle reaches an intersection, it randomly decides to turn or go straight.
#  5. If a vehicle hits the map boundary, it turns around.
#
import pandas as pd
import random
import numpy as np
from tqdm import tqdm # A progress bar for the simulation

print("🛠️ Starting mobility data generation using pure Python...")

# ==============================================================================
# STEP 1: DEFINE SIMULATION PARAMETERS (CUSTOMIZE HERE!)
# ==============================================================================
class Config:
    # World Parameters
    DURATION_SECONDS = 3600
    NUM_USERS = 50
    GRID_SIZE_METERS = 10000
    INTERSECTION_SPACING = 1000 # Defines the road grid, e.g., roads every 1km
    TIMESTEP_SECONDS = 1

    # Vehicle Parameters
    MIN_SPEED_MPS = 11 # ~40 km/h
    MAX_SPEED_MPS = 20 # ~72 km/h

    # Output file
    OUTPUT_FILENAME = "user_mobility.csv"

# ==============================================================================
# STEP 2: CREATE A VEHICLE CLASS TO MANAGE MOVEMENT
# ==============================================================================
class Vehicle:
    def __init__(self, vehicle_id, config):
        self.id = f"veh{vehicle_id}"
        self.config = config

        # Place the vehicle at a random intersection to start
        self.x = random.choice(range(0, config.GRID_SIZE_METERS + 1, config.INTERSECTION_SPACING))
        self.y = random.choice(range(0, config.GRID_SIZE_METERS + 1, config.INTERSECTION_SPACING))
        self.z = 0.0

        # Assign a random speed and initial direction (North, East, South, West)
        self.speed = random.uniform(config.MIN_SPEED_MPS, config.MAX_SPEED_MPS)
        self.direction = random.choice([(0, 1), (1, 0), (0, -1), (-1, 0)]) # (dx, dy)

    def is_at_intersection(self):
        """Check if the vehicle is at a grid intersection."""
        return self.x % self.config.INTERSECTION_SPACING == 0 and \
               self.y % self.config.INTERSECTION_SPACING == 0

    def choose_new_direction(self):
        """At an intersection, decide whether to turn or go straight."""
        # Simple logic: 60% chance to go straight, 20% left, 20% right
        # Get current direction index (0=N, 1=E, 2=S, 3=W)
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        current_idx = directions.index(self.direction)

        choice = random.random()
        if choice < 0.6: # Go straight
            new_idx = current_idx
        elif choice < 0.8: # Turn right
            new_idx = (current_idx + 1) % 4
        else: # Turn left
            new_idx = (current_idx - 1 + 4) % 4

        self.direction = directions[new_idx]

    def move(self):
        """Update the vehicle's position for one timestep."""
        # Check for intersection and potentially change direction
        if self.is_at_intersection():
            self.choose_new_direction()

        # Move the vehicle based on its current direction and speed
        dx, dy = self.direction
        self.x += dx * self.speed * self.config.TIMESTEP_SECONDS
        self.y += dy * self.speed * self.config.TIMESTEP_SECONDS

        # Boundary check: if vehicle goes off the map, make it turn back
        if not (0 <= self.x <= self.config.GRID_SIZE_METERS):
            self.direction = (-dx, dy)
            self.x = np.clip(self.x, 0, self.config.GRID_SIZE_METERS)

        if not (0 <= self.y <= self.config.GRID_SIZE_METERS):
            self.direction = (dx, -dy)
            self.y = np.clip(self.y, 0, self.config.GRID_SIZE_METERS)

    def get_state(self, timestep):
        """Return the current state as a dictionary."""
        return {
            'timestep': timestep,
            'vehicle_id': self.id,
            'x': round(self.x, 2),
            'y': round(self.y, 2),
            'z': self.z,
            'speed': round(self.speed, 2)
        }

# ==============================================================================
# STEP 3: RUN THE MAIN SIMULATION LOOP
# ==============================================================================
# Initialize all vehicles
config = Config()
vehicles = [Vehicle(i, config) for i in range(config.NUM_USERS)]

# Store all data points
mobility_data = []

# Main loop
print(f"🚗 Simulating {config.NUM_USERS} users for {config.DURATION_SECONDS} seconds...")
for t in tqdm(range(1, config.DURATION_SECONDS + 1)):
    for vehicle in vehicles:
        vehicle.move()
        mobility_data.append(vehicle.get_state(t))

# ==============================================================================
# STEP 4: CREATE AND SAVE THE DATAFRAME
# ==============================================================================
print("\n🔄 Converting data and saving to CSV...")

# Create DataFrame
df = pd.DataFrame(mobility_data)

# Reorder columns to match the required format
df = df[['timestep', 'vehicle_id', 'x', 'y', 'z', 'speed']]

# Save to CSV
df.to_csv(config.OUTPUT_FILENAME, index=False)

print(f"\n🎉 SUCCESS! Your data has been generated and saved as '{config.OUTPUT_FILENAME}'.")
print(f"File contains {len(df)} rows of data for {df['vehicle_id'].nunique()} unique users.")
print("You can now find it in the file browser on the left and download it.")

🛠️ Starting mobility data generation using pure Python...
🚗 Simulating 50 users for 3600 seconds...


100%|██████████| 3600/3600 [00:02<00:00, 1474.19it/s]



🔄 Converting data and saving to CSV...

🎉 SUCCESS! Your data has been generated and saved as 'user_mobility.csv'.
File contains 180000 rows of data for 50 unique users.
You can now find it in the file browser on the left and download it.
